In [ ]:
!pip install transformers

In [ ]:
 # !wget -nc https://github.com/edrees2022/NLP-With-Transformers/blob/main/notebooks/Edrees/sentimernt_anlaysis_pipline/AirlineTweets.csv

In [11]:
from transformers import pipeline

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split



In [ ]:
# Basic usage
classifier = pipeline("sentiment-analysis")

In [ ]:
# output is a dictionary
classifier("this is such a great movie!")

In [ ]:
type(classifier)

In [ ]:
classifier("this show was not interesting")

In [ ]:
classifier("this show was not interesting")

In [ ]:
classifier("this show was interesting")

In [ ]:
# prompt: get me list of sentences that positive and negative and set on my classifier

classifier( ["This course is just what I need", "The weather is great today!", "I'm feeling so happy!", "This show was not interesting", "The food was terrible", "I'm feeling so sad"])


In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
classifier = pipeline("sentiment-analysis", device=0)

In [43]:
df_ = pd.read_csv("AirlineTweets.csv")

In [ ]:
df_.head()

In [44]:
df=df_[['airline_sentiment', 'text']].copy()

In [ ]:
df['airline_sentiment'].hist()

In [46]:
df=df[df.airline_sentiment != 'neutral'].copy()

In [47]:
target_map={'positive':1, 'negative':0}
df['target']= df['airline_sentiment'].map(target_map)

In [ ]:
df.head()

In [ ]:
len(df)

In [65]:
texts = df['text'].tolist()
predictions = classifier(texts)

In [51]:
predictions

[{'label': 'POSITIVE', 'score': 0.6070840954780579},
 {'label': 'NEGATIVE', 'score': 0.9973449110984802},
 {'label': 'NEGATIVE', 'score': 0.9995823502540588},
 {'label': 'NEGATIVE', 'score': 0.9854250550270081},
 {'label': 'POSITIVE', 'score': 0.9553212523460388},
 {'label': 'POSITIVE', 'score': 0.9827088117599487},
 {'label': 'POSITIVE', 'score': 0.9994851350784302},
 {'label': 'POSITIVE', 'score': 0.9993509650230408},
 {'label': 'POSITIVE', 'score': 0.9940821528434753},
 {'label': 'POSITIVE', 'score': 0.9863133430480957},
 {'label': 'POSITIVE', 'score': 0.9996304512023926},
 {'label': 'NEGATIVE', 'score': 0.9980512857437134},
 {'label': 'POSITIVE', 'score': 0.9964436888694763},
 {'label': 'NEGATIVE', 'score': 0.9968666434288025},
 {'label': 'POSITIVE', 'score': 0.9787647724151611},
 {'label': 'POSITIVE', 'score': 0.9995218515396118},
 {'label': 'NEGATIVE', 'score': 0.9954516291618347},
 {'label': 'POSITIVE', 'score': 0.9995624423027039},
 {'label': 'POSITIVE', 'score': 0.999874830245

In [66]:
probs = [d['score'] if d['label'].startswith('p') else 1 - d['score'] \
         for d in predictions]
probs

[0.39291590452194214,
 0.0026550889015197754,
 0.0004176497459411621,
 0.014574944972991943,
 0.04467874765396118,
 0.01729118824005127,
 0.0005148649215698242,
 0.0006490349769592285,
 0.005917847156524658,
 0.013686656951904297,
 0.0003695487976074219,
 0.001948714256286621,
 0.0035563111305236816,
 0.0031333565711975098,
 0.021235227584838867,
 0.0004781484603881836,
 0.004548370838165283,
 0.0004375576972961426,
 0.0001251697540283203,
 0.004880666732788086,
 0.00040477514266967773,
 0.0006322264671325684,
 0.015428304672241211,
 0.01828092336654663,
 0.0004482865333557129,
 0.0023748278617858887,
 0.006551921367645264,
 0.0023932456970214844,
 0.1338280439376831,
 0.0003237724304199219,
 0.15843814611434937,
 0.0032317638397216797,
 0.00021535158157348633,
 0.1761431097984314,
 0.0006104707717895508,
 0.09156662225723267,
 0.004861712455749512,
 0.00021797418594360352,
 0.0014235973358154297,
 0.004673957824707031,
 0.00464242696762085,
 0.019041001796722412,
 0.00883018970489502,

In [56]:
preds = [1 if d['label'].startswith('P') else 0 for d in predictions]

preds = np.array(preds)

In [ ]:
print("acc:", np.mean(df['target']== preds))

In [ ]:

#  compute the confusion matrix will set normalize the true son that each row sums to one
cm = confusion_matrix(df['target'], preds, normalize= 'true')
cm

In [ ]:
# draw heatmap
# Scikit-Learn is transitioning to v1 but it's not available on colab
# The changes modify how confusion matrices are plotted

def plot_cm(cm):
  classes =['negative', 'positive']
  df_cm =pd.DataFrame(cm, index=classes, columns=classes)
  ax=sn.heatmap(df_cm, annot=True, fmt='g')
  ax.set_xlabel("Predicted")
  ax.set_ylabel("Target")

plot_cm(cm)

In [ ]:
f1_score(df['target'], preds)

In [ ]:
f1_score(1-df['target'], 1- preds)